# Creando una Vista estatica
- Vistas basadas en funciones
- Vistas basadas en clase

## Cómo crear una aplicación en Django

python manage.py startapp nombreapp

python manga.py starapp ecommerce

- funciones se nombran en minúsculas
- clases se nombran comenzando con mayúsculas

# ecommerce/views.py

In [ ]:
from django.shortcuts import render
from django.http import HttpResponse
# Create your views here.

def home(request):
    html = """
    <!DOCTYPE html>
    <html>
        <head>
            <title>Ecommerce</title>
            <style>
                h1 {color:green}
            </style>
        </head>
        <body>
            <h1>Hola Mundo</h1>
        </body>
    </html>
"""
    return HttpResponse(html)

# ecommerce/urls.py

In [ ]:
from django.urls import path

from ecommerce import views

urlpatterns = [
    path("", views.home, name="home"),
]

# config/urls.py

In [ ]:
"""
URL configuration for hello project.

The `urlpatterns` list routes URLs to views. For more information please see:
    https://docs.djangoproject.com/en/5.0/topics/http/urls/
Examples:
Function views
    1. Add an import:  from my_app import views
    2. Add a URL to urlpatterns:  path('', views.home, name='home')
Class-based views
    1. Add an import:  from other_app.views import Home
    2. Add a URL to urlpatterns:  path('', Home.as_view(), name='home')
Including another URLconf
    1. Import the include() function: from django.urls import include, path
    2. Add a URL to urlpatterns:  path('blog/', include('blog.urls'))
"""
from django.contrib import admin
from django.urls import include
from django.urls import path

urlpatterns = [
    path("up/", include("up.urls")),
    path("", include("pages.urls")),
    path("ecommerce/", include("ecommerce.urls")), #<---------------------
    path("admin/", admin.site.urls),
    path("__debug__/", include("debug_toolbar.urls")),
]

# Respuesta http y redireccionamiento

Middleware - procesos internos entre un request y un response

In [ ]:
def home(request):
    html = """
    <!DOCTYPE html>
    <html>
        <head>
            <title>Ecommerce</title>
            <style>
                h1 {color:green}
            </style>
        </head>
        <body>
            <h1>Hola Mundo</h1>
        </body>
    </html>
"""
    return HttpResponse(html)
# def home(request):
#     response = HttpResponse()
#     response.write('<p>Página no encontrada</p>')
#     response.status_code = 404
#     return response

def redirect_test(request):
    return HttpResponseRedirect("/ecommerce/")

## CRUD y Vistas

Forma dinamica de usar las vistas es el CRUD

- Crear modelos
- Agregarlos a INSTALLED_APPS
- Crear y aplicar migraciones
- Agregarlos 

## ecommerce/models.py

In [ ]:
class ProductModel(models.Model):
    title = models.TextField()
    price = models.FloatField()

## config/settings.py

In [ ]:
INSTALLED_APPS = [
    "pages.apps.PagesConfig",
    "ecommerce.apps.EcommerceConfig",
    "debug_toolbar",
    "django.contrib.admin",
    "django.contrib.auth",
    "django.contrib.contenttypes",
    "django.contrib.sessions",
    "django.contrib.messages",
    "django.contrib.staticfiles",
]

## Como crear las migraciones

- python manage.py makemigrations
- python manage.py migrate

## Como registrar un modelo en admin

In [ ]:
from django.contrib import admin
from .models import ProductModel
# Register your models here.
admin.site.register(ProductModel)

## Crear un super usuario para la base de datos
- python manage.py createsuperuser

## Tipos básicos de vistas

- List view: Donde ves todos los usuarios creados
- Create view: Donde puedes insertar datos para crear usuarios
- Retrive y Update: Si le doy clic a un usuario y lo modifico
- Deleted view: Botón rojo para eliminar

## ecommerce/views.py

In [ ]:
from django.shortcuts import render
from django.http import HttpResponse,HttpResponseRedirect
from .models import ProductModel
# Create your views here.

def product_model_list_view(request):
    queryset = ProductModel.objects.all()
    print(queryset)
    return HttpResponse("Un ecommerce personalizado")

## Usando templates

## ecommerce/views.py

In [ ]:
from django.shortcuts import render
from django.http import HttpResponse,HttpResponseRedirect
from .models import ProductModel
# Create your views here.

def product_model_list_view(request):
    queryset = ProductModel.objects.all()
    print(queryset)
    template = "ecommerce/list-view.html"
    context = {}
    return render(request, template, context)

# templates/ecommerce/list-view.html

In [ ]:
<h1>
    Vista de listado
</h1>

# Usando el contexto

In [ ]:
from django.shortcuts import render
from django.http import HttpResponse,HttpResponseRedirect
from .models import ProductModel
# Create your views here.

def product_model_list_view(request):
    queryset = ProductModel.objects.all()
    print(queryset)
    template = "ecommerce/list-view.html"
    context = {
        "products":queryset
    }
    return render(request, template, context)

# templates/ecommerce/list-view.html

In [ ]:
<h1>
    Vista de listado
</h1>

{% for product in products %}
<li>
    {{product.title}} {{product.price}}
</li>
{% endfor %}

# Protegiendo la vista

In [ ]:
from django.shortcuts import render
from django.http import HttpResponse,HttpResponseRedirect
from .models import ProductModel
# Create your views here.

def product_model_list_view(request):
    queryset = ProductModel.objects.all()
    template = "ecommerce/list-view.html"
    context = {
        "products":queryset
    }
    if request.user.is_authenticated:
        template = "ecommerce/list-view.html"
    else:
        template = 'ecommerce/list-view-public.html'
    return render(request, template, context)

# La mejor forma es usando decoradores

In [ ]:
from django.contrib.auth.decorators import login_required
from django.shortcuts import render
from django.http import HttpResponse,HttpResponseRedirect
from .models import ProductModel
# Create your views here.

@login_required(login_url="/admin/login") #En configuración global se pude usar la variable LOGIN_URL con la ruta al login para no pasar el argumento
def product_model_list_view(request):
    queryset = ProductModel.objects.all()
    template = "ecommerce/list-view.html"
    context = {
        "products":queryset
    }
    if request.user.is_authenticated:
        template = "ecommerce/list-view.html"
    else:
        template = 'ecommerce/list-view-public.html'
    return render(request, template, context)

# Vista de detalles

## ecommerce/view.py

In [ ]:
def product_model_detail_view(request,product_id):
    instance = get_object_or_404(ProductModel, id=product_id)
    context = {
        "product":instance
    }
    template = "ecommerce/detail-view.html"
    return render(request,template,context)

## ecommerce/urls.py

In [ ]:
from django.urls import path

from ecommerce import views

urlpatterns = [
    path("", views.product_model_list_view, name="home"),
    path("<int:product_id>", views.product_model_detail_view, name="detail"),
]

# templates/ecommerce/detail-view.html

In [ ]:
<h1>
    {{product.title}}
    {{product.price}}
</h1>

## templates/ecommerce/list-view.html

In [ ]:
<h1>
    Vista de listado
</h1>

{% for product in products %}
<li>
    <a href="/ecommerce/{{product.id}}">{{product.title}}</a>
</li>
{% endfor %}

# Vista de creación

## ecommerce/create-view.py

In [ ]:
def product_model_create_view(request):
    form = ProductModelForm(request.POST or None)
    if form.is_valid():
        instance = form.save(commit=False)
        instance.save()
        messages.success(request,"Producto creado con éxito")
        return HttpResponseRedirect("/ecommerce/{product_id}".format(product_id=instance.id))
    context = {
        "form":form
    }
    template = "ecommerce/create-view.html"
    return render(request,template,context)

## ecommerce/forms.py

In [ ]:
from django import forms
from .models import ProductModel

class ProductModelForm(forms.ModelForm):
    class Meta:
        model = ProductModel
        fields = [
            "title",
            "price"
        ]

## templates/ecommerce/messages.html

In [ ]:
{% if messages %}
<ul class="messages">
    {% for message in messages %}
    <li{% if message.tags %} class="{{ message.tags }}"{% endif %}>{{ message }}</li>
    {% endfor %}
</ul>
{% endif %}

## templates/ecommerce/create-view.html

In [ ]:
{%include "ecommerce/messages.html"%}
<h1>
    Creando nuevo producto
</h1>
<form action="" method="post">
    {% csrf_token%}
    {{ form.as_p}}
    <input type="submit" value="Crear">
</form>

# Agregar la función {%include "ecommerce/messages.html"%} en todos los templates

## ecommerce/urls.py

In [ ]:
from django.urls import path

from ecommerce import views

urlpatterns = [
    path("", views.product_model_list_view, name="home"),
    path("<int:product_id>", views.product_model_detail_view, name="detail"),
    path("create/", views.product_model_create_view, name="create"),
]

# Vista de actualización

## ecommerce/update-view.py

In [ ]:
def product_model_create_view(request,product_id=None):
    instance = get_object_or_404(ProductModel, id=product_id)
    form = ProductModelForm(request.POST or None,instance=instance)
    if form.is_valid():
        instance = form.save(commit=False)
        instance.save()
        messages.success(request,"Producto actualizado con éxito")
        return HttpResponseRedirect("/ecommerce/{product_id}".format(product_id=instance.id))
    context = {
        "form":form
    }
    template = "ecommerce/update-view.html"
    return render(request,template,context)


## templates/ecommerce/update-view.html

In [ ]:
<h1>
    Actualización de producto {{form.instance.title}}
</h1>
{{form.instance.title}}
<form action="" method="post">
    {% csrf_token%}
    {{ form.as_p}}
    <input type="submit" value="Actualizar">
</form>

## ecommerce/urls.py

In [ ]:
from django.urls import path

from ecommerce import views

urlpatterns = [
    path("", views.product_model_list_view, name="home"),
    path("<int:product_id>", views.product_model_detail_view, name="detail"),
    path("create/", views.product_model_create_view, name="create"),
    path("<int:product_id>/edit/", views.product_model_update_view, name="update"),
]

# Vista para eliminar

## ecommerce/delete-view.py

In [ ]:
def product_model_delete_view(request,product_id):
    instance = get_object_or_404(ProductModel, id=product_id)
    if request.method == 'POST':
        instance.delete()
        HttpResponseRedirect("/ecommerce/")
        messages.success(request,'Producto eliminado')
        return HttpResponseRedirect("/ecommerce/")
    context = {
        "product":instance
    }
    template = "ecommerce/delete-view.html"
    return render(request,template,context)

## templetes/commerce/delete-view.html

In [ ]:
{%include "ecommerce/messages.html"%}
<h1>
    Eliminar {{product.title}}
</h1>

<form action="" method="post">
    {% csrf_token%}
    ¿Estas seguro que deseas eliminar el producto?
    <input type="submit" value="Eliminar">
    <a href="/ecommerce/{{product.id}}">Cancelar</a>
</form>

## ecommerce/urls.py

In [ ]:
from django.urls import path

from ecommerce import views

urlpatterns = [
    path("", views.product_model_list_view, name="home"),
    path("<int:product_id>", views.product_model_detail_view, name="detail"),
    path("create/", views.product_model_create_view, name="create"),
    path("<int:product_id>/edit/", views.product_model_update_view, name="update"),
    path("<int:product_id>/delete/", views.product_model_delete_view, name="delete"),
]

# Agregando un buscador

## ecommerce/view.py

In [ ]:
def product_model_list_view(request):
    query = request.GET.get("q",None)
    queryset = ProductModel.objects.all()
    if query is not None:
        queryset = queryset.filter(
            Q(title__icontains=query) |
            Q(price__icontains=query)
        )
    template = "ecommerce/list-view.html"
    context = {
        "products":queryset
    }
    if request.user.is_authenticated:
        template = "ecommerce/list-view.html"
    else:
        template = 'ecommerce/list-view-public.html'
    return render(request, template, context)

## templete/ecommerce/search.html

In [ ]:
<form action="/ecommerce/" method="get">
    <input type="text" name="q" placeholder="Buscar">
</form>

## templete/ecommerce/list-view.html

In [ ]:
{% include "ecommerce/search.html" %}
<h1>
    Vista de listado
</h1>

{% for product in products %}
<li>
    <a href="/ecommerce/{{product.id}}">{{product.title}}</a>
</li>
{% endfor %}